# Css Analysis Youtube

In [1]:
import pandas as pd

In [3]:
data = pd.read_csv("comments_youtube.csv")


/var/folders/jl/654jv9hd1gjd2cryn_0qfrqc0000gn/T/ipykernel_10015/3894868132.py:1: DtypeWarning: Columns (6,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("comments_youtube.csv")


In [5]:
data


,video_id,comment_id,author,text,published_at,like_count,published_at_x,like_count_x,title,description,published_at_y,channel_title,view_count,like_count_y,comment_count
0,YM-vc6JqzhM,UgxUOZLngoiMfd0iMWR4AaABAg,@Aaaaaaaaliyaahhh,They should have shot him as they do to innoce...,2024-10-29T20:18:59Z,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,YM-vc6JqzhM,UgyJb16uXbJIKjePq0x4AaABAg,@Lunasherbertbunny,It is his disorder because my best friend has ...,2024-01-10T22:05:27Z,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,YM-vc6JqzhM,UgwKUTfSyk180lqsKmt4AaABAg,@mzubuki,"Proverbs 1:7 ""For the fear of the lord is the ...",2024-01-05T12:29:30Z,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,YM-vc6JqzhM,Ugx9JtiCH8HsqKjNt454AaABAg,@iansoberano5103,I think this Cruz kid should have gotten the d...,2023-12-14T01:45:52Z,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,YM-vc6JqzhM,UgwSvEFkN1molO7B_V54AaABAg,@МатвейМалинин-щ4ф,Отморозь.,2023-11-30T10:34:07Z,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195927,lVMcVs0Ty1E,Ugws4OeayzqaqW-HLVV4AaABAg,@AutomatedPersonnelUnit_3947,"I'd have liked to have ""Probed"" that useless w...",2022-10-05T15:33:16Z,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
195928,lVMcVs0Ty1E,UgzlbV4wuvInIR2xYAx4AaABAg,@filipecoutinho5706,Execute it already,2022-10-05T15:29:34Z,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
195929,lVMcVs0Ty1E,UgyBSMDw3L4qfd1gieR4AaABAg,@sacredbanana,lol he thinks H2O is hydrogen and helium,2022-10-05T15:27:38Z,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
195930,lVMcVs0Ty1E,UgyA4OwByjTUD18197l4AaABAg,@joshx022,Please notify me when he's being injected. Th...,2022-10-05T15:18:03Z,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
